# 🧠 Clasificación, Limpieza y Evaluación de Datos con Explicaciones Completas

## 📂 Cargar base de datos desde ruta verificada

In [ ]:
import pandas as pd
import os

ruta = './datos/dataset_ejemplo_1300.csv'
if not os.path.exists(ruta):
    raise FileNotFoundError(f'❌ El archivo no se encuentra en: {ruta}')
df_raw = pd.read_csv(ruta)
df_raw.head()

## 🎯 Crear variables `Alta_conectividad` y `Nivel_conectividad`

In [ ]:
df_raw['Alta_conectividad'] = (df_raw['Horas_Internet'] > 3.5).astype(int)

def clasificar_conectividad(horas):
    if horas <= 1:
        return 0
    elif horas <= 3.5:
        return 1
    else:
        return 2

df_raw['Nivel_conectividad'] = df_raw['Horas_Internet'].apply(clasificar_conectividad)

## 🧼 Imputación de valores faltantes con `SimpleImputer`

| Estrategia         | Qué hace                                       | Cuándo usarla                                     | Ejemplo                                                                 |
|--------------------|-----------------------------------------------|---------------------------------------------------|-------------------------------------------------------------------------|
| `'mean'`           | Sustituye con la media                        | Datos numéricos sin outliers                      | Edad: `[25, 28, NaN, 22, 30] → NaN = 26.25`                             |
| `'median'`         | Sustituye con la mediana                      | Datos numéricos con outliers                      | Ingreso: `[1000, 1200, NaN, 8000, 1100] → NaN = 1150`                   |
| `'most_frequent'`  | Sustituye con el valor más común (moda)       | Datos categóricos                                 | Género: `['F', 'M', NaN, 'F'] → NaN = 'F'`                              |
| `'constant'`       | Sustituye con un valor definido por el usuario| Para normalizar o identificar nulos               | Ciudad: `['Bogotá', NaN] → fill_value='Desconocida' → NaN = 'Desconocida'`

In [ ]:
from sklearn.impute import SimpleImputer

df_clean = df_raw.copy()
imputer_edad = SimpleImputer(strategy='median')
df_clean['Edad'] = imputer_edad.fit_transform(df_clean[['Edad']])
imputer_ingreso = SimpleImputer(strategy='mean')
df_clean['Ingreso'] = imputer_ingreso.fit_transform(df_clean[['Ingreso']])

## 🔣 Codificación de variables categóricas

In [ ]:
df_clean = pd.get_dummies(df_clean, drop_first=True)

## 📏 Escalado con `MinMaxScaler`

In [ ]:
from sklearn.preprocessing import MinMaxScaler
cols_numericas = df_clean.select_dtypes(include=['int64', 'float64']).columns
scaler = MinMaxScaler()
df_clean[cols_numericas] = scaler.fit_transform(df_clean[cols_numericas])

## ✅ Filtrar solo columnas numéricas antes del modelo

In [ ]:
Xc = df_clean.drop(columns=['Alta_conectividad'])
Xc = Xc.select_dtypes(include=['int64', 'float64'])
yc = df_clean['Alta_conectividad']

## 🤖 Entrenamiento del modelo final

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.3, random_state=42)
model_after = LogisticRegression(max_iter=200)
model_after.fit(Xc_train, yc_train)
yc_pred = model_after.predict(Xc_test)
acc_after = accuracy_score(yc_test, yc_pred)
print('✅ Exactitud del modelo:', round(acc_after, 4))

## 📊 Evaluación del modelo con métricas adicionales

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print("📌 Matriz de confusión:")
print(confusion_matrix(yc_test, yc_pred))

print("\n📌 Reporte de clasificación:")
print(classification_report(yc_test, yc_pred))

print("\n📌 Distribución de clases en y_test:")
print(yc_test.value_counts(normalize=True))

## 🧪 Validación cruzada

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(LogisticRegression(max_iter=200), Xc, yc, cv=5)
print("📈 Resultados de validación cruzada:", scores)
print("Promedio:", scores.mean())